Look into tf.data
Look at TF traceviewer (available outside TPU?)
Goal is to get from IO bound to CPU xor better GPU bound (GPU is the most expensive device)
look at Derek Murrays tf.data talk
accelerator: GPU, TPU etc.
Goal is to pipeline loading (disk), preprocessing (cpu) and loading data into GPU (PCI?)

In [ ]:
# ResNet-50: simple input pipeline

def input_fn(batch_size, data_dir):
    files = tf.data.Dataset.list_files(data_dir)
    dataset = tf.data.TFRecordDataset(files, num_parallel_read=32)
    # num_parallel_read: TFRecordDataset uses tf.contrib.data.parallel_interleave under the hood
    dataset = dataset.shuffle(2048) # sliding window of 2048 records
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.map(parser_fn, num_parallel_calls=64)
    # parser_fn:
    # 1. parse TF.Example
    # 2. image decode
    # 3. random crop and flip
    # 4. color transformations
    #
    # num_parallel_calls: paralleize across multiple cores
    
    dataset = dataset.batch(batch_size)
    
    dataset = dataset.prefetch(2)
    # prefetch makes sure everything above is pipelined with the GPU training
    # larger prefetch buffers also smooth over latency variability
    
    return dataset

Advanced techniques 
 * use fused dataset operators
   * map-and-batch
   * shuffle-and-repeat
 * enable sloppy interleave
 * adjust read buffer sizes
 * perform transformation on accelerators
 
 TODO include optimized-input-pipeline image

use more compact formats:
most projects use:
fp32: single precision IEEE floating point format

alternative:
fp16: half-precision IEEE floating poin format

More compact format allows bigger model on the GPU. Also GPUs are usually IO-bound and not compute bound, here smaller datasize helps as well.

Look at bfloat16

Hardware developments:
NVIDIA volta's tensor cores have hardware accelerated matrix operations

Image layout affects performance:
 * channel last (NHWC)
 * channel first (NCHW) - FASTER!!!

For best performance use latest TF and driver versions
Look at TensorRT

TODO: include micro-benchmarks image

TODO: look at XLA: xcelerated linear algebra